<a href="https://colab.research.google.com/github/aceking007/MLproject/blob/master/Copy_of_MS16124_coursework2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# NEURAL NETWORK ASSIGNMENT
## Name : Arpit Omprakash
## Roll number: MS16124


In [0]:
import numpy as np
import pandas as pd

# Problem 1
def load_data(filepath):
    df = pd.read_csv(filepath, header=None) # header=None assigns integer headers starting at 0
    # df = df[0:20]
    # print (df.head())
    Y = df.iloc[:,784] # Just the last column
    list_columns = list(range(df.shape[1])) # Makes a list of indices
    list_columns.remove(784) # Removes the last column from list
    X = df.iloc[:,list_columns] # All data except last column
    X = X/255 # Normalization
    # Arpit Omprakash
    return X, Y

X_train, Y_train = load_data("https://raw.githubusercontent.com/aceking007/MLproject/master/mnist_train.csv")

print(X_train.head())
print(Y_train.head())
print(X_train.shape, Y_train.shape)

In [0]:
# Problem 2 (No hidden layer)

np.random.seed(42)

# Softmax activation function for the whole output - carries out softmax in each row
def softmax(x):
    lis = []
    for logits in x:
        exps = [np.exp(logit-np.max(logits)) for logit in logits]
        exp_sum = sum(exps)
        out = [j/exp_sum for j in exps]
        lis.append(out)
    return np.asarray(lis)

# Cross entropy function
# The epsilon is used to clip extreme values and thus prevents the loss from being infinite 
def cross_entropy(predictions, targets, epsilon=1e-12):
    predictions = np.clip(predictions, epsilon, 1. - epsilon)
    n = predictions.shape[0]
    # Arpit Omprakash 
    ce = -np.sum(targets*np.log(predictions+1e-9))/n
    return ce

# Initialization
def initialize_perceptron(X):
    input_neurons = X.shape[1]
    output_neurons = 10
    weights = np.random.randn(input_neurons,output_neurons)
    bias = np.random.randn(output_neurons)
    return weights, bias

def update_weights_perceptron(X,Y,weights,bias,lr):
    # Feedforward
    zoutput = np.dot(X,weights) + bias
    predictions = softmax(zoutput)
    # Backpropagation
    target = np.asarray(pd.get_dummies(Y))
    # Printing the loss value
    print("Loss = ",cross_entropy(predictions,target))
    n = predictions.shape[0]
    dcost_dzo = np.asarray(predictions-target)
    dcost_weights_output = np.dot(X.T,dcost_dzo)
    dcost_bias_output = np.asarray(np.sum(dcost_dzo,axis=0)/n)
    # Arpit Omprakash
    # Updating weights and bias
    updated_weights = weights-lr*dcost_weights_output
    updated_bias = bias-lr*dcost_bias_output
    return updated_weights, updated_bias

weights, bias = initialize_perceptron(X_train)
for i in range(500):
    print ('Epoch #',i)
    weights,bias = update_weights_perceptron(X_train,Y_train,weights,bias,lr = 0.001)


In [0]:
# Problem 3 (Single hidden layer with N neurons)

np.random.seed(42)

# Softmax activation function for the whole output - carries out softmax in each row
def softmax(x):
    lis = []
    for logits in x:
        exps = [np.exp(logit-np.max(logits)) for logit in logits]
        exp_sum = sum(exps)
        out = [j/exp_sum for j in exps]
        lis.append(out)
    return np.asarray(lis)

# Cross entropy function
# The epsilon is used to clip extreme values and thus prevents the loss from being infinite 
def cross_entropy(predictions, targets, epsilon=1e-12):
    predictions = np.clip(predictions, epsilon, 1. - epsilon)
    n = predictions.shape[0]
    ce = -np.sum(targets*np.log(predictions+1e-9))/n
    return ce

# Sigmoid function
def sigmoid(x):
    return 1/(1+np.exp(-x))

# Derivative of sigmoid function
def sigmoid_der(x):
    return sigmoid(x) *(1-sigmoid (x))

# Initialization
def initialize_single_layer(X, hidden_layer_neurons):
    input_neurons = X.shape[1]
    output_neurons = 10
    weights_hiddenlayer = np.random.randn(input_neurons,hidden_layer_neurons)
    bias_hiddenlayer = np.random.randn(hidden_layer_neurons)
    weights_outputlayer = np.random.randn(hidden_layer_neurons,output_neurons)
    bias_outputlayer = np.random.randn(output_neurons)
    weights = [weights_hiddenlayer,weights_outputlayer]
    bias = [bias_hiddenlayer,bias_outputlayer]
    return weights, bias    

def update_weights_single_layer(X,Y,weights,bias,lr):
    # Feedforward
    # For the hidden layer
    zhidden = np.dot(X,weights[0]) + bias[0]
    ahidden = sigmoid(zhidden)
    # For the output layer
    zoutput = np.dot(ahidden,weights[1]) + bias[1]
    aoutput = softmax(zoutput)
    # Backpropagation
    target = np.asarray(pd.get_dummies(Y))
    predictions = aoutput
    # Printing the loss value
    print("Loss = ",cross_entropy(predictions,target))
    # For the output layer
    dcost_dzo = predictions - target
    dzo_dwo = ahidden
    dcost_weights_output = np.dot(dzo_dwo.T,dcost_dzo)
    dcost_bias_output = dcost_dzo
    # For the hidden layer
    # Arpit Omprakash
    dzo_dah = weights[1]
    dcost_dah = np.dot(dcost_dzo, dzo_dah.T)
    dah_dzh = sigmoid_der(zhidden)
    dcost_dzh = dah_dzh * dcost_dah
    dzh_dwh = X
    dcost_weights_hidden = np.dot(dzh_dwh.T, dcost_dzh)
    dcost_bias_hidden = dcost_dzh
    # Updating weights and biases
    updated_weights = [weights[0]-lr*dcost_weights_hidden,weights[1]-lr*dcost_weights_output]
    updated_bias = [bias[0]-lr*dcost_bias_hidden,bias[1]-lr*dcost_bias_output]
    return updated_weights,updated_bias


weights, bias = initialize_single_layer(X_train,hidden_layer_neurons = 12)
for i in range(500):
    print ('Epoch #',i)
    weights,bias = update_weights_single_layer(X_train,Y_train,weights,bias,lr = 0.001)

In [0]:
# Problem 4 (Two hidden layers with N neurons each)

np.random.seed(42)

# Softmax activation function for the whole output - carries out softmax in each row
def softmax(x):
    lis = []
    for logits in x:
        exps = [np.exp(logit-np.max(logits)) for logit in logits]
        exp_sum = sum(exps)
        out = [j/exp_sum for j in exps]
        lis.append(out)
    return np.asarray(lis)

# Cross entropy function
# The epsilon is used to clip extreme values and thus prevents the loss from being infinite 
def cross_entropy(predictions, targets, epsilon=1e-12):
    predictions = np.clip(predictions, epsilon, 1. - epsilon)
    n = predictions.shape[0]
    ce = -np.sum(targets*np.log(predictions+1e-9))/n
    return ce

# Sigmoid function
def sigmoid(x):
    return 1/(1+np.exp(-x))

# Derivative of sigmoid function
def sigmoid_der(x):
    return sigmoid(x) *(1-sigmoid (x))

# Initialization
def initialize_double_layer(X, hidden_layer_neurons):
    input_neurons = X.shape[1]
    output_neurons = 10
    weights_h1 = np.random.randn(input_neurons,hidden_layer_neurons)
    bias_h1 = np.random.randn(hidden_layer_neurons)
    weights_h2 = np.random.randn(hidden_layer_neurons,hidden_layer_neurons)
    bias_h2 = np.random.randn(hidden_layer_neurons)
    weights_o = np.random.randn(hidden_layer_neurons,output_neurons)
    bias_o = np.random.randn(output_neurons)
    weights = [weights_h1, weights_h2,weights_o]
    bias = [bias_h1, bias_h2, bias_o]
    return weights, bias

def update_weights_double_layer(X,Y,weights,bias,lr):
    # Feedforward
    # For h1 layer (hidden)
    zh1 = np.dot(X,weights[0]) + bias[0]
    ah1 = sigmoid(zh1)
    # For h2 layer(hidden)
    zh2 = np.dot(ah1,weights[1]) + bias[1]
    ah2 = sigmoid(zh2)
    # For output layer
    zo = np.dot(ah2,weights[2]) + bias[2]
    ao = softmax(zo)
    # Backpropagation
    target = np.asarray(pd.get_dummies(Y))
    predictions = ao
    # Printing loss value
    # Arpit Omprakash
    print("Loss: ", cross_entropy(predictions,target))
    # For the output layer
    dcost_dzo = predictions - target
    dzo_dwo = ah2
    dcost_weights_o = np.dot(dzo_dwo.T,dcost_dzo)
    dcost_bias_o = dcost_dzo
    # For h2 layer (hidden)
    dzo_dah2 = weights[2]
    dcost_dah2 = np.dot(dcost_dzo , dzo_dah2.T)
    dah2_dzh2 = sigmoid_der(zh2)
    dcost_dzh2 = dah2_dzh2 * dcost_dah2
    dzh2_dwh2 = ah1
    dcost_weights_h2 = np.dot(dzh2_dwh2.T, dcost_dzh2)
    dcost_bias_h2 = dcost_dzh2
    # For h1 layer (hidden)
    dzh2_dah1 = weights[1]
    dcost_dah1 = np.dot(dcost_dzh2, dzh2_dah1.T)
    dah1_dzh1 = sigmoid_der(zh1)
    dcost_dzh1 = dah1_dzh1 * dcost_dah1
    dzh1_dwh1 = X
    dcost_weights_h1 = np.dot(dzh1_dwh1.T, dcost_dzh1)
    dcost_bias_h1 = dcost_dzh1
    # Updating weights and biases
    updated_weights = [weights[0]-lr*dcost_weights_h1,weights[1]-lr*dcost_weights_h2,weights[2]-lr*dcost_weights_o]
    updated_bias = [bias[0]-lr*dcost_bias_h1,bias[1]-lr*dcost_bias_h2,bias[2]-lr*dcost_bias_o]
    return updated_weights,updated_bias

weights, bias = initialize_double_layer(X_train, hidden_layer_neurons = 10)
for i in range(500):
    print ('Epoch #',i)
    weights,bias = update_weights_double_layer(X_train,Y_train,weights,bias,lr = 0.001)

In [0]:
# Problem 5 (Two hidden layers with activation function)

np.random.seed(42)

# Softmax activation function for the whole output - carries out softmax in each row
def softmax(x):
    lis = []
    for logits in x:
        exps = [np.exp(logit-np.max(logits)) for logit in logits]
        exp_sum = sum(exps)
        out = [j/exp_sum for j in exps]
        lis.append(out)
    return np.asarray(lis)

# Cross entropy function
# The epsilon is used to clip extreme values and thus prevents the loss from being infinite 
def cross_entropy(predictions, targets, epsilon=1e-12):
    predictions = np.clip(predictions, epsilon, 1. - epsilon)
    n = predictions.shape[0]
    ce = -np.sum(targets*np.log(predictions+1e-9))/n
    return ce

# Sigmoid function
def sigmoid(x):
    return 1/(1+np.exp(-x))

# Derivative of sigmoid function
def sigmoid_der(x):
    return sigmoid(x) *(1-sigmoid (x))

# TanH function
def tanh(x):
    temp = 2/(1+np.exp(-2*x))
    return temp - 1

# Derivative of TanH function
def tanh_der(x):
    return 1 - (tanh(x)*tanh(x))

# ReLU function
def relu(x):
    return x * (x > 0) 

# Derivative of ReLU function
def relu_der(x):
    return 1. * (x > 0)

# References to functions and derivatives
f1 = sigmoid
f2 = tanh
f3 = relu
f = [f1, f2, f3]
d1 = sigmoid_der
d2 = tanh_der
d3 = relu_der
d = [d1, d2, d3]

# Exception class for any error in the parameters provided
class ParameterError(Exception):
    pass


# Initialization
def initialize_double_layer_act(X, hidden_layer_neurons):
    input_neurons = X.shape[1]
    output_neurons = 10
    weights_h1 = np.random.randn(input_neurons,hidden_layer_neurons)
    bias_h1 = np.random.randn(hidden_layer_neurons)
    weights_h2 = np.random.randn(hidden_layer_neurons,hidden_layer_neurons)
    bias_h2 = np.random.randn(hidden_layer_neurons)
    weights_o = np.random.randn(hidden_layer_neurons,output_neurons)
    bias_o = np.random.randn(output_neurons)
    weights = [weights_h1, weights_h2,weights_o]
    bias = [bias_h1, bias_h2, bias_o]
    return weights, bias

def update_weights_double_layer_act(X,Y,weights,bias,lr,activation):
    # Changing activation function according to the parameter
    if activation == 'sigmoid':
        func = f1
        deriv = d1
    elif activation == 'tanh':
        func = f2
        deriv = d2
    elif activation == 'relu':
        func = f3
        deriv = d3
    else:
        print("Invalid activation function")
        raise ParameterError('Activation function incorrect')
    # Feedforward
    # For h1 layer (hidden)
    zh1 = np.dot(X,weights[0]) + bias[0]
    ah1 = func(zh1)
    # For h2 layer(hidden)
    zh2 = np.dot(ah1,weights[1]) + bias[1]
    ah2 = func(zh2)
    # For output layer
    zo = np.dot(ah2,weights[2]) + bias[2]
    ao = softmax(zo)
    # Backpropagation
    target = np.asarray(pd.get_dummies(Y))
    predictions = ao
    # Printing loss value
    print("Loss: ", cross_entropy(predictions,target))
    # For the output layer
    # Arpit Omprakash
    dcost_dzo = predictions - target
    dzo_dwo = ah2
    dcost_weights_o = np.dot(dzo_dwo.T,dcost_dzo)
    dcost_bias_o = dcost_dzo
    # For h2 layer (hidden)
    dzo_dah2 = weights[2]
    dcost_dah2 = np.dot(dcost_dzo , dzo_dah2.T)
    dah2_dzh2 = deriv(zh2)
    dcost_dzh2 = dah2_dzh2 * dcost_dah2
    dzh2_dwh2 = ah1
    dcost_weights_h2 = np.dot(dzh2_dwh2.T, dcost_dzh2)
    dcost_bias_h2 = dcost_dzh2
    # For h1 layer (hidden)
    dzh2_dah1 = weights[1]
    dcost_dah1 = np.dot(dcost_dzh2, dzh2_dah1.T)
    dah1_dzh1 = deriv(zh1)
    dcost_dzh1 = dah1_dzh1 * dcost_dah1
    dzh1_dwh1 = X
    dcost_weights_h1 = np.dot(dzh1_dwh1.T, dcost_dzh1)
    dcost_bias_h1 = dcost_dzh1
    # Updating weights and biases
    updated_weights = [weights[0]-lr*dcost_weights_h1,weights[1]-lr*dcost_weights_h2,weights[2]-lr*dcost_weights_o]
    updated_bias = [bias[0]-lr*dcost_bias_h1,bias[1]-lr*dcost_bias_h2,bias[2]-lr*dcost_bias_o]
    return updated_weights,updated_bias

weights, bias = initialize_double_layer_act(X_train,hidden_layer_neurons = 10)
for i in range(500):
    print ('Epoch #',i)
    try:
      weights,bias = update_weights_double_layer_act(X_train,Y_train,weights,bias,lr = 0.001,activation = 'sigmoid')
    except ParameterError as e:
      break

In [0]:
# Problem 6 (Two hidden layers with activation function and momentum)

np.random.seed(42)

# Softmax activation function for the whole output - carries out softmax in each row
def softmax(x):
    lis = []
    for logits in x:
        exps = [np.exp(logit-np.max(logits)) for logit in logits]
        exp_sum = sum(exps)
        out = [j/exp_sum for j in exps]
        lis.append(out)
    return np.asarray(lis)

# Cross entropy function
# The epsilon is used to clip values make the np.log(predictions) infinitely large
def cross_entropy(predictions, targets, epsilon=1e-12):
    predictions = np.clip(predictions, epsilon, 1. - epsilon)
    n = predictions.shape[0]
    ce = -np.sum(targets*np.log(predictions+1e-9))/n
    return ce

# Sigmoid function
def sigmoid(x):
    return 1/(1+np.exp(-x))

# Derivative of sigmoid function
def sigmoid_der(x):
    return sigmoid(x) *(1-sigmoid (x))

# TanH function
def tanh(x):
    temp = 2/(1+np.exp(-2*x))
    return temp - 1

# Derivative of TanH function
def tanh_der(x):
    return 1 - (tanh(x)*tanh(x))

# ReLU function
def relu(x):
    return x * (x > 0) 

# Derivative of ReLU function
def relu_der(x):
    return 1. * (x > 0)

# References to functions and derivatives
f1 = sigmoid
f2 = tanh
f3 = relu
f = [f1, f2, f3]
d1 = sigmoid_der
d2 = tanh_der
d3 = relu_der
d = [d1, d2, d3]

# Exception class for any error in the parameters provided
class ParameterError(Exception):
    pass


# Initialization
def initialize_double_layer_act(X, hidden_layer_neurons):
    input_neurons = X.shape[1]
    output_neurons = 10
    weights_h1 = np.random.randn(input_neurons,hidden_layer_neurons)
    bias_h1 = np.random.randn(hidden_layer_neurons)
    weights_h2 = np.random.randn(hidden_layer_neurons,hidden_layer_neurons)
    bias_h2 = np.random.randn(hidden_layer_neurons)
    weights_o = np.random.randn(hidden_layer_neurons,output_neurons)
    bias_o = np.random.randn(output_neurons)
    weights = [weights_h1, weights_h2,weights_o]
    bias = [bias_h1, bias_h2, bias_o]
    return weights, bias

# Main implementation
def update_weights_double_layer_act(X,Y,weights,bias,lr,activation,momentum):
    global prev_dcost_weights_o, prev_dcost_bias_o, prev_dcost_weights_h2, prev_dcost_bias_h2, prev_dcost_weights_h1, prev_dcost_bias_h1
    # Changing activation function according to the parameter
    if activation == 'sigmoid':
        func = f1
        deriv = d1
    elif activation == 'tanh':
        func = f2
        deriv = d2
    elif activation == 'relu':
        func = f3
        deriv = d3
    else:
        print("Invalid activation function")
        raise ParameterError('Activation function incorrect')
    ###### Feedforward
    ### For h1 layer (hidden)
    zh1 = np.dot(X,weights[0]) + bias[0]
    ah1 = func(zh1)
    ### For h2 layer(hidden)
    zh2 = np.dot(ah1,weights[1]) + bias[1]
    ah2 = func(zh2)
    ### For output layer
    zo = np.dot(ah2,weights[2]) + bias[2]
    ao = softmax(zo)
    ###### Backpropagation
    target = np.asarray(pd.get_dummies(Y))
    predictions = ao
    # Printing loss value
    print("Loss: ", cross_entropy(predictions,target))
    ### For the output layer
    dcost_dzo = predictions - target
    dzo_dwo = ah2
    # Updating weights in accordance with previous weights using momentum
    dcost_weights_o = momentum*prev_dcost_weights_o + (1-momentum)*np.dot(dzo_dwo.T,dcost_dzo)
    dcost_bias_o = momentum*prev_dcost_bias_o + (1-momentum)*dcost_dzo
    # Changing previous weights to current weights
    prev_dcost_weights_o = dcost_weights_o
    prev_dcost_bias_o = dcost_bias_o
    ### For h2 layer (hidden)
    dzo_dah2 = weights[2]
    dcost_dah2 = np.dot(dcost_dzo , dzo_dah2.T)
    dah2_dzh2 = deriv(zh2)
    dcost_dzh2 = dah2_dzh2 * dcost_dah2
    dzh2_dwh2 = ah1
    # Updating weights in accordance with previous weights using momentum
    dcost_weights_h2 = momentum*prev_dcost_weights_h2 + (1-momentum)*np.dot(dzh2_dwh2.T, dcost_dzh2)
    dcost_bias_h2 = momentum*prev_dcost_bias_h2 + (1-momentum)*dcost_dzh2
    # Changing previous weights to current weights
    prev_dcost_weights_h2 = dcost_weights_h2
    prev_dcost_bias_h2 = dcost_bias_h2
    ### For h1 layer (hidden)
    dzh2_dah1 = weights[1]
    dcost_dah1 = np.dot(dcost_dzh2, dzh2_dah1.T)
    dah1_dzh1 = deriv(zh1)
    dcost_dzh1 = dah1_dzh1 * dcost_dah1
    dzh1_dwh1 = X
    # Updating weights in accordance with previous weights using momentum
    # Arpit Omprakash
    dcost_weights_h1 = momentum*prev_dcost_weights_h1 + (1-momentum)*np.dot(dzh1_dwh1.T, dcost_dzh1)
    dcost_bias_h1 = momentum*prev_dcost_bias_h1 + (1-momentum)*dcost_dzh1
    # Changing previous weights to current weights
    prev_dcost_weights_h1 = dcost_weights_h1
    prev_dcost_bias_h1 = dcost_bias_h1
    ###### Updating weights and biases
    updated_weights = [weights[0]-lr*dcost_weights_h1,weights[1]-lr*dcost_weights_h2,weights[2]-lr*dcost_weights_o]
    updated_bias = [bias[0]-lr*dcost_bias_h1,bias[1]-lr*dcost_bias_h2,bias[2]-lr*dcost_bias_o]
    return updated_weights,updated_bias

# Function that can takes epoch value and trains the neural network
def training_nn(X,Y,weights,bias,lr,activation,momentum,epochs = 10):
    for epoch in range(epochs):
        print ('Epoch #', epoch)
        try:
          weights,bias = update_weights_double_layer_act(X,Y,weights,bias,lr,activation,momentum)
        except ParameterError:
          break
    return weights

# Variables to store previous gradients (for implementing momentum)
prev_dcost_weights_o = 0
prev_dcost_bias_o = 0
prev_dcost_weights_h2 = 0
prev_dcost_bias_h2 = 0
prev_dcost_weights_h1 = 0
prev_dcost_bias_h1 = 0


################################################################################
# Initializing weights and bias
weights, bias = initialize_double_layer_act(X_train,hidden_layer_neurons = 10)
# Training the model for "epoch" number of epochs
# The training returns the resulting weights after training
training_nn(X_train, Y_train, weights, bias, lr = 0.01, activation = 'sigmoid', momentum = 0.9, epochs = 1000)